## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,50.00,53,75,18.41,broken clouds
1,1,Tiksi,RU,71.6872,128.8694,8.31,69,100,7.99,overcast clouds
2,2,Paamiut,GL,61.9940,-49.6678,33.91,76,5,5.95,clear sky
3,3,Ban Nahin,LA,18.2418,104.2122,73.31,88,28,2.89,scattered clouds
4,4,Espumoso,BR,-28.7247,-52.8497,65.03,94,94,14.41,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
selected_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
selected_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Rikitea,PF,-23.1203,-134.9692,78.46,73,6,7.78,light rain
8,8,Najran,SA,17.4924,44.1277,77.00,18,20,6.26,few clouds
9,9,Sinnamary,GF,5.3833,-52.9500,78.06,84,62,4.07,broken clouds
16,16,Matamoros,MX,25.5333,-103.2500,90.00,12,75,5.75,broken clouds
18,18,Bathsheba,BB,13.2167,-59.5167,78.80,69,75,13.80,volcanic ash
19,19,Saint-Georges,GD,12.0564,-61.7485,82.40,74,20,9.22,few clouds
25,25,Vila Velha,BR,-20.3297,-40.2925,77.00,78,0,4.61,clear sky
27,27,Labuhan,ID,-6.8844,112.2051,77.61,86,63,4.90,broken clouds
31,31,Culiacan,MX,24.7994,-107.3897,84.00,44,75,11.50,broken clouds
34,34,Namibe,AO,-15.1961,12.1522,79.45,69,62,3.11,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
selected_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
selected_cities_df.notnull().sum()

City_ID                217
City                   217
Country                217
Lat                    217
Lng                    217
Max Temp               217
Humidity               217
Cloudiness             217
Wind Speed             217
Current Description    217
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# no empty rows found

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = selected_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Rikitea,PF,78.46,light rain,-23.1203,-134.9692,
8,Najran,SA,77.00,few clouds,17.4924,44.1277,
9,Sinnamary,GF,78.06,broken clouds,5.3833,-52.9500,
16,Matamoros,MX,90.00,broken clouds,25.5333,-103.2500,
18,Bathsheba,BB,78.80,volcanic ash,13.2167,-59.5167,
19,Saint-Georges,GD,82.40,few clouds,12.0564,-61.7485,
25,Vila Velha,BR,77.00,clear sky,-20.3297,-40.2925,
27,Labuhan,ID,77.61,broken clouds,-6.8844,112.2051,
31,Culiacan,MX,84.00,broken clouds,24.7994,-107.3897,
34,Namibe,AO,79.45,broken clouds,-15.1961,12.1522,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")
        pass

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Rikitea,PF,78.46,light rain,-23.1203,-134.9692,Pension Maro'i
8,Najran,SA,77.00,few clouds,17.4924,44.1277,محمد هادي ال فطيح
9,Sinnamary,GF,78.06,broken clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
16,Matamoros,MX,90.00,broken clouds,25.5333,-103.2500,Quinta Belem
18,Bathsheba,BB,78.80,volcanic ash,13.2167,-59.5167,Atlantis Hotel
...,...,...,...,...,...,...,...
701,Belmonte,BR,77.00,few clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
707,Buin,CL,82.40,clear sky,-33.7333,-70.7500,Paihuen
708,Dingle,PH,78.31,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
712,Lafiagi,NG,85.05,scattered clouds,8.8667,5.4167,Bus group international


In [18]:
hotel_df.count()

City                   217
Country                217
Max Temp               217
Current Description    217
Lat                    217
Lng                    217
Hotel Name             217
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))